<a href="https://colab.research.google.com/github/aubreymoore/crb-damage-detector-colab/blob/main/detect_and_annotate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# detect_and_annotate.ipynb

NOTE: The following documentation is already slightly out of date.
Please visit https://github.com/aubreymoore/crb-damage-detector-colab before running this notebook for the first time.

This Colab Jupyter notebook runs a custom YOLOv8 object detector which scans images to find three object classes: live coconut palms, dead coconut palms and v-shaped cuts symptomatic of damage caused by coconut rhinoceros beetle, *Oryctes rhinoceros*.

IMPORTANT: Shortly after the MAIN PROGRAM section begins executing, a BROWSE button will appear below the active cell to allow you to upload single file of input data from your loacal machine to Colab.

**Note that Colab will just sit there and not do anything until you have entered a path to a test file of URLs or a ZIP file of images on your local machine.** [Click here to scroll down to the "Browser" button.](#scrollTo=5zSjfTXvIv2q&line=1&uniqifier=1)

You may choose between 2 options:
* A TEXT file (\*.txt) containing URLs for images to be scanned. One URL per line. (This is the most efficient option.)
* A ZIP file (\*.zip) containing images to be scanned.


Test data are available in a companion GitHub repository at https://github.com/aubreymoore/crb-damage-detector-colab. To use the test data, download it to your local computer it as a [ZIP file](https://github.com/aubreymoore/crb-damage-detector-colab/archive/refs/heads/main.zip) and unzip it. If you have **git** installed, you can clone the repo as an alternative. The TEXT file or ZIP file to be uploaded to Colab will be found in the repository's **data** folder.

To scan images, select **Runtime | Run all** on the main menu.
Results will be in a temporary OUTPUT folder which you can access using the **File browser** in the left Colab panel.

When image scanning is complete, the OUTPUT folder will be compressed into a single ZIP file and automatically downloaded to your computer.

### TODO

- [ finished 2024-10-19] Reduce size of images in the companion GH repo to max dimension of 960px
- [ ] Copy current trained model to companion GH repo
- [ ] Copy this Jupyter notebook to companion GH repo
- [ ] Add confidence values to bounding box labels.
- [ ] Add database to OUTPUT folder
- [ ] Extract GPS coordinates from image files
- [ ] Figure out how to use URLs to access images stored on OneDrive (Sharepoint)

# Load Python packages which are not preinstalled by Colab

In [17]:
%pip install ultralytics -q
%pip install supervision -q
%pip install imutils -q
%pip install icecream -q
%pip install ipython-autotime -q

time: 14.3 s (started: 2024-10-22 22:52:58 +00:00)


# Import modules

In [18]:
import cv2
import supervision as sv
from ultralytics import YOLO
import imutils
import glob
import os
import shutil
from skimage import io
from icecream import ic
from google.colab import files
# ultralytics.checks()

time: 622 µs (started: 2024-10-22 22:53:12 +00:00)


# Load cell timer

In [19]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 778 µs (started: 2024-10-22 22:53:12 +00:00)


# Define functions

In [20]:
def upload_model_weights():
  '''
  Upload model weights from GitHub repo to **weights.pt** only if this file does not already exist.
  '''
  !wget -nc https://github.com/aubreymoore/code-for-CRB-damage-ai/raw/refs/heads/main/models/3class/train5/weights/best.pt -O weights.pt

# upload_model_weights()

time: 547 µs (started: 2024-10-22 22:53:12 +00:00)


In [21]:
def load_model_weights():
  model = YOLO('weights.pt')

time: 538 µs (started: 2024-10-22 22:53:12 +00:00)


In [22]:
def create_input_folder():
  if not os.path.exists('INPUT'):
    os.makedirs('INPUT')

# create_input_folder()

time: 619 µs (started: 2024-10-22 22:53:12 +00:00)


In [23]:
def create_output_folder():
  if not os.path.exists('OUTPUT'):
    os.makedirs('OUTPUT')

# create_output_folder()

time: 531 µs (started: 2024-10-22 22:53:12 +00:00)


In [24]:
def run_garbage_disposal():
  '''
  Delete any data files left over from the last run.
  '''
  shutil.rmtree('INPUT', ignore_errors=True)
  shutil.rmtree('OUTPUT', ignore_errors=True)
  try:
    os.remove('OUTPUT.zip')
    os.remove('weights.pt')
    os.remove('sample_data')
  except:
    ic('Could not remove one or more files.')
    pass

# run_garbage_disposal()

time: 501 µs (started: 2024-10-22 22:53:12 +00:00)


# Upload images (\*.zip) or list of URLs (\*.txt)

When this cell runs, a **Browse** button will appear beneath it.
Click on the button and navigate to the file you wan to upload.
If you have downloaded the example data from the companion GitHub repo, try
uploading something like **/home/aubrey/Desktop/crb-damage-detector-colab/data/urls.txt** or **/home/aubrey/Desktop/crb-damage-detector-colab/data/resized.zip**.


In [25]:
def upload_and_unpack_zip_or_txt():

  uploaded = files.upload(target_dir='INPUT')
  filename = list(uploaded.keys())[0]

  urls = None
  image_file_dir = None

  if filename.endswith('.txt'):
    input_mode = 'text'
    with open(filename, 'r') as f:
      urls = f.read().splitlines()
  elif filename.endswith('.zip'):
    input_mode = 'zip'
    !unzip -q $filename -d INPUT
    image_file_dir = f'INPUT/{filename}'.replace('.zip', '')
    ic(image_file_dir)
  else:
    raise ValueError('INPUT file must be *.txt or *.zip.')
  return input_mode, urls, image_file_dir

# input_mode, urls, image_file_dir = upload_and_unpack_zip_or_txt()
# ic(input_mode)
# ic(urls)
# ic(image_file_dir)

time: 669 µs (started: 2024-10-22 22:53:12 +00:00)


In [26]:
def get_input_file_list():
  return glob.glob(f'INPUT/**/*', recursive=True)

# get_input_file_list()

time: 326 µs (started: 2024-10-22 22:53:12 +00:00)


In [27]:
def detect_objects(image, model, box_annotator, label_annotator, csv_sink):
  '''
  detect objects in and image
  returns an annotated image
  '''
  results = model(image)[0]
  detections = sv.Detections.from_ultralytics(results)
  # ic(detections)
  annotated_image = box_annotator.annotate(image, detections=detections)
  labels = [f"{model.model.names[class_id]} {confidence:.2f}" for class_id, confidence in zip(detections.class_id, detections.confidence)]
  annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)
  return detections, annotated_image

# csv_sink = sv.CSVSink('detections.csv')
# csv_sink.open()

# upload_model_weights()
# model = YOLO('weights.pt')
# box_annotator = sv.BoxAnnotator()
# label_annotator = sv.LabelAnnotator()

# url = 'https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/Vanuatu_July_2022_Sulav/resized-images/IMG_0532.JPG?raw=true'
# image = imutils.url_to_image(url)
# detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink)
# ic(detections)
# sv.plot_image(annotated_image)

# custom_data = {'url': url}
# csv_sink.append(detections, custom_data)

# csv_sink.close()

time: 577 µs (started: 2024-10-22 22:53:12 +00:00)


# MAIN PROGRAM

In [28]:
# Clear data files from previous run
run_garbage_disposal()

create_input_folder()
create_output_folder()

# Upload images or list of URLs
input_mode, urls, image_file_dir = upload_and_unpack_zip_or_txt()
ic(input_mode)
ic(urls)
ic(image_file_dir)

# If model is not defined upload weights from trained model and load them
try:
  model
except NameError:
  upload_model_weights()
  model = YOLO('weights.pt')

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

csv_sink = sv.CSVSink('OUTPUT/detections.csv')
csv_sink.open()

# Scan images


if input_mode == 'text':
  for url in urls:
    try:
      detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink)
      csv_sink.append(detections, custom_data={'url': url,'image_path': image_path})


      # image = imutils.url_to_image(url)
      # results = model(image)[0]
      # detections = sv.Detections.from_ultralytics(results)
      # annotated_image = box_annotator.annotate(image, detections=detections)
      # labels = [f"{model.model.names[class_id]} {confidence:.2f}" for class_id, confidence in zip(detections.class_id, detections.confidence)]
      # annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)

      # sv.plot_image(annotated_image)

      # Extract filename from URL
      filename = url.split('/')[-1]
      pos = filename.find('?')
      if pos >= 0:
        filename = filename[:pos]

      output_path = f'OUTPUT/{filename}'.replace('.', '_annotated.')
      ic(output_path)
      os.makedirs(os.path.dirname(output_path), exist_ok = True)
      cv2.imwrite(output_path, annotated_image)
    except:
      print(f'Error processing {url}')
    continue

if input_mode == 'zip':
  input_file_list = get_input_file_list()
  ic(input_file_list)
  for image_path in input_file_list:
    ic(image_path)
    try:
      image = cv2.imread(image_path)
      detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink)
      csv_sink.append(detections, custom_data={'url': '', 'image_path': image_path})

      # ic(type(image))
      # results = model(image)[0]
      # detections = sv.Detections.from_ultralytics(results)
      # labels = [f"{model.model.names[class_id]} {confidence:.2f}" for class_id, confidence in zip(detections.class_id, detections.confidence)]
      # annotated_image = box_annotator.annotate(image, detections=detections)
      # annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)

      # sv.plot_image(annotated_image)

      filename = os.path.basename(image_path)
      output_path = f'OUTPUT/{filename}'.replace('.', '_annotated.')
      ic(output_path)
      os.makedirs(os.path.dirname(output_path), exist_ok = True)
      result = cv2.imwrite(output_path, annotated_image)
      ic(result)
    except:
      print(f'Error processing {image_path}')
    continue

csv_sink.close()

ic| 'Could not remove one or more files.'


KeyboardInterrupt: 

time: 41 s (started: 2024-10-22 22:53:12 +00:00)


## Please click on the Browse buttom when it appears above this cell.

### Download OUTPUT folder as a ZIP file

In [ ]:
!zip -r OUTPUT.zip OUTPUT

In [ ]:
from google.colab import files
files.download("OUTPUT.zip")

# FINISHED
If everything worked as intended, you should find a file named **OUTPUT.zip** in your Downloads folder. Unzip this file to see results.

In [ ]:
print('FINISHED')